### Introduction to Localisation
Localisation allows a car to know precisely where it is.
* Answers Q: 'Where is our car in a given map with an accuracy of 3-10cm?'
    * GPS is not precise enough (accuracy of the width of a lane, 1-3m. Sometimes as broad as 10-15m.)
    * Robot takes info about environment and compares it to what it knows about the real world (a map).
    * Intuition: Reducing uncertainty about location using information e.g. seeing the Eiffel Tower.
* Use onboard sensors (lidar, radar) to measure distance to static obstacles and bearings in the local coordinate system of our car. Some obstacles that are observed by onboard sensors are also on the map (which has a global coordinate system.)
    * Need to match observations with map information (transform both coordinates) with high precision. 
(9.1, 9.2)    

![](images/9.1.png)

![](images/9.2.png)

Lesson Outline
* What is Localisation?
* Bayesian Filters: 1D Example in C++
* Motion Models
* 2D Particle Filter in C++

# Localisation


## What is localisation?

![](images/10.1.png)

* Measurement of door transforms our belief function: increased belief of locations next to the door, decreased belief of locations not next to the door.
* Third dist: Assume robot moves to the right by a certain distance. Shift the belief to the right by a certain dist (but peaks are flatter).
* Fourth dist: Sees door again after a small amout of rightward movement.

## Uniform Dist code

In [ ]:
#  Modify your code to create probability vectors, p, of arbitrary 
#  size, n. Use n=5 to verify that your new solution matches 
#  the previous one.

n=5
p=[1.0/n for i in range(n)]

print(p)

### Updating probabilities after sensing

![](images/10.2.png)
* Multiply belief it's in a red cell by e.g. 0.6, multiply belief it's in a green cell by 0.2.
* Normalise by sum of values to make posterior a valid probability distribution.


### Program localisation function in Python

In [ ]:
# Localisation function in Python

p=[0.2, 0.2, 0.2, 0.2, 0.2]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2

def sense(p, Z):
    q=[]
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss))
    s = sum(q)
    for i in range(len(q)):
        q[i] = q[i] / s
    return q

for Z in measurements:
    p = sense(p, Z)
print(p)


### Program exact and inaccurate motion in Python
* Exact motion case


In [ ]:
#Program a function that returns a new distribution 
#q, shifted to the right by U units. If U=0, q should 
#be the same as p.

p=[0, 1, 0, 0, 0]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2

def sense(p, Z):
    q=[]
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss))
    s = sum(q)
    for i in range(len(q)):
        q[i] = q[i] / s
    return q

def move(p, U):
    #
    #ADD CODE HERE
    #
    U %= len(p)
    q = p[-U:] + p[:-U]
    // or
    // for i in range(len(p)):
    //     q.append(p[(i-U) % len(p)])
    return q

print move(p, 1)


* Inaccurate motion case
![](images/10.3.png)


In [ ]:
#Modify the move function to accommodate the added 
#probabilities of overshooting or undershooting 
#the intended destination.

p=[0, 1, 0, 0, 0]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

def sense(p, Z):
    q=[]
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss))
    s = sum(q)
    for i in range(len(q)):
        q[i] = q[i] / s
    return q

def move(p, U):
    q = []
    for i in range(len(p)):
        q.append(p[(i-U)%len(p)]*pExact + p[(i-U-1)%len(p)]*pUndershoot + p[(i-U+1)%len(p)]*pOvershoot)
    return q
    

print move(p, 1)


### Limit distribution
* Uniform dist: maximal uncertainty, min information. every time we move we lose information (uncertainty increases).
* Balance property: P(X) breakdown must hold in the limit as when things don't move any more. (?)

![](images/10.4.png)

In [ ]:
#write code that moves 1000 times and then prints the 
#resulting probability distribution.
# p approx equals [0.2, 0.2, 0.2, 0.2, 0.2]

p=[0, 1, 0, 0, 0]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

def sense(p, Z):
    q=[]
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss))
    s = sum(q)
    for i in range(len(q)):
        q[i] = q[i] / s
    return q

def move(p, U):
    q = []
    for i in range(len(p)):
        s = pExact * p[(i-U) % len(p)]
        s = s + pOvershoot * p[(i-U-1) % len(p)]
        s = s + pUndershoot * p[(i-U+1) % len(p)]
        q.append(s)
    return q
#
# ADD CODE HERE
#
for i in range(1000):
    p = move(p, 1)
print p


### Localisation is cycling through 'Sense and Move'

![](images/10.5.png)

* Entropy: Measure of information Expected log-likelihood. 
    * Update step (move) makes entropy go down (lose info), measurement step (sense) makes entropy go up (gain info).



In [ ]:
#Given the list motions=[1,1] which means the robot 
#moves right and then right again, compute the posterior 
#distribution if the robot first senses red, then moves 
#right one, then senses green, then moves right again, 
#starting with a uniform prior distribution.

p=[0.2, 0.2, 0.2, 0.2, 0.2]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
motions = [1,1]
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

def sense(p, Z):
    q=[]
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss))
    s = sum(q)
    for i in range(len(q)):
        q[i] = q[i] / s
    return q

def move(p, U):
    q = []
    for i in range(len(p)):
        s = pExact * p[(i-U) % len(p)]
        s = s + pOvershoot * p[(i-U-1) % len(p)]
        s = s + pUndershoot * p[(i-U+1) % len(p)]
        q.append(s)
    return q
#
# ADD CODE HERE
#
for i in range(len(motions)):
    p = sense(p, measurements[i])
    p = move(p, motions[i])
print p         


### Summary
![](images/10.6.png)
![](images/10.7.png)

* Move: Convolution: Reverse engineer the situation.

### Αppendix
#### Formalising localisation

![](images/10.8.png)
![](images/10.9.png)
* P(Z) technically probability of observing dist without location information.
* Alpha is normaliser
